In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('data/входящие.xlsx')

df = df.fillna(method='ffill')

df_mail = df[df['Контактное лицо'].str.contains('E-mail') == True]
df_tel = df[df['Контактное лицо'].str.contains('Тел.') == True]
df_name = df[df['Контактное лицо'].str.contains('Тел.' or 'E-mail')==False]
df_final = df.merge(df_mail[['Наименование компании', 'Контактное лицо']], on = 'Наименование компании', how='left')
df_final = df_final.merge(df_tel[['Наименование компании', 'Контактное лицо']], on = 'Наименование компании', how='left')
df_final = df_final.merge(df_name[['Наименование компании', 'Контактное лицо']], on = 'Наименование компании', how='left')

df_final = df_final.drop_duplicates(subset='Наименование компании')
df_final.to_excel(excel_writer='new.xlsx')

MergeError: Passing 'suffixes' which cause duplicate columns {'Контактное лицо_x'} is not allowed.

In [ ]:
df_final = df_final.drop_duplicates(subset='Наименование компании')
display(df_final.shape)
df_final.to_excel(excel_writer='new.xlsx')

(188, 8)

In [ ]:
mass_fin = pd.read_excel('data/Консолидированный эксель контакты.xlsx')
display(mass_fin.head())

,ФАЙЛ,Название компании,Контактные данные ответственного за ВЭД лица
0,1,"Акционерное общество ""Производственное объедин...",не заполнено
1,2,"ООО фирма ""Дизельсервис""",не заполнено
2,3,"ООО СК ""Сибирь-Строй""","Никитина Наталья Геннадьевна, тел. 8-913-060-3..."
3,4,"ООО ПКФ ""Атлантис-Пак""",Шевцова Александра Вадимовна shevtsova_av@atla...
4,5,АО Медисорб,"Коваленко Александр Анатольевич, a.kovalenko@m..."


In [10]:
mass_fin_with_nans = pd.read_excel('data/Консолидированный эксель контакты.xlsx')

def get_email(data):
    """Функция для выделения email.

    Args:
        data (str): Название столбца в dataframe

    Returns:
        str: 1 или несколько emailов, разделенные точкой с запятой
        Numpy.NaN: Пропуск, если emailа нет
    """
    sign_list = ['\n', ':', ',', '(', ')', ';', '/', '>', '<', '«', '»', '"']
    data = str(data)
    for sign in sign_list:
        data = data.replace(sign, ' ')
    data = data.split(' ')
    
    result = []
    for value in data:
        if '@' in value:
            result.append(value)
    if len(result) == 1:
        return str(result[0])
    elif len(result) > 1:
        final_result = ''
        for adress in result:
            final_result += adress+';'+' '
        return final_result
    elif len(result) == 0:
        return np.nan

mass_fin_with_nans['Email'] = mass_fin_with_nans['Контактные данные ответственного за ВЭД лица '].apply(get_email)
display(mass_fin_with_nans.head())
mass_fin_with_nans.to_excel(excel_writer='mass_fin.xlsx')

mass_fin_without_nulls = mass_fin_with_nans.copy()
mass_fin_without_nulls = mass_fin_without_nulls[mass_fin_without_nulls['Email'].notna() == True]
mass_fin_without_nulls.to_excel(excel_writer='mass_fin_without_nulls.xlsx')

display(mass_fin_with_nans.iloc[554][3])

display(f"Число непустых имейлов - {mass_fin_with_nans[mass_fin_with_nans['Email'].notna() == True]['Email'].count()}")

        

,ФАЙЛ,Название компании,Контактные данные ответственного за ВЭД лица,Email
0,1,"Акционерное общество ""Производственное объедин...",не заполнено,NaN
1,2,"ООО фирма ""Дизельсервис""",не заполнено,NaN
2,3,"ООО СК ""Сибирь-Строй""","Никитина Наталья Геннадьевна, тел. 8-913-060-3...",plan-03@csib.info
3,4,"ООО ПКФ ""Атлантис-Пак""",Шевцова Александра Вадимовна shevtsova_av@atla...,shevtsova_av@atlantis-pak.ru
4,5,АО Медисорб,"Коваленко Александр Анатольевич, a.kovalenko@m...",a.kovalenko@medisorb.ru


'custom@unichel.ru; minx74ru@mail.ru; '

'Число непустых имейлов - 450'